# 1. Protein Analysis Using `ProtParam`

In [11]:
from Bio import Entrez
from Bio import SeqIO
from Bio.SeqUtils import ProtParam

Entrez.email = 'nguyuling@graduate.utm.my'
try:
    handle = Entrez.efetch(
        db='protein',
        id='P10308',
        rettype='fasta',
        retmode='text'
    )
    record = SeqIO.read(handle, 'fasta')
    handle.close()
except Exception as e:
    print(f'Error fetching seqeunce: {e}')
    

def analyse_protein(record):
    analysis = ProtParam.ProteinAnalysis(str(record.seq))
    properties = {
        'Protein ID' : record.id,
        'Sequence' : record.seq,
        'Length' : f"{analysis.length} amino acids",
        'Molecular Weight': f"{round(analysis.molecular_weight(),4)} Da",
        'Isoelectric Point': round(analysis.isoelectric_point(),4),
        'Instability Index': round(analysis.instability_index(),4),
        'Aromaticity': round(analysis.aromaticity(),4),
        'Gravy': round(analysis.gravy(),4)
    }
    return properties

properties = analyse_protein(record)
for key, value in properties.items():
    print(f'{key:<25}: {value}')
    

Protein ID               : sp|P10308.1|FIBER_BPT3
Sequence                 : MANVIKTVLTYQLDGSNRDFNIPFEYLARKFVVVTLIGVDRKVLSINADYRFATRTTISLTKAWGPADGYTTIELRRVTSTTDQLVDFTDGSILRAYDLNVAQIQTMHVAEEARDLTADTIGVNNDGHLDARGRRIVNLANAVDDRDAVPFGQLKTMNQNSWQARNEALQFRNEAETFRNQTEVFKNESGTNATNTKQWRDEANGSRDEAEQFKNTAGQYATSAGNSATTATQSEVNAENSATDADNSRNLAEQHADRLELEADKLGIFNGLAGRIDKGDGTNVYWKGGIHANGRLYLTSDGFDCGQYQQFFGGSAGRYSVMEWGIEKAWLMHVERRERTTAIVDNIQLVVNGHIIAQGGDMTGPLKLQNGHALYLESASDKAQYILSKDGNRNNWHIGRGSDNNNDCTFHSYVYGTNLTLKPDYAVVNKRFHVGQAVVATDGNIQGTKWGRKWLDAYLNDTYVKKTMAWTQVWAAASDSYMGGGSQTDTLHRTCDSATYGLRPETTIGTSSELVLTVSTSFQPRRWLKFQIHSNGRVFKNIADRAATPTAIAVEDV
Length                   : 557 amino acids
Molecular Weight         : 61800.607 Da
Isoelectric Point        : 5.9905
Instability Index        : 27.0564
Aromaticity              : 0.0898
Gravy                    : -0.5506


# 2. Comparing Protein Family Members

In [15]:
from Bio import Entrez
from Bio import SeqIO
from Bio.SeqUtils import ProtParam
import pandas as pd
import time

Entrez.email = 'nguyuling@graduate.utm.my'

proteins = ['P01308', 'P01322', 'P01325', 'P01329']
records = []
processed = 0
current   = 0

# Fetch all the protein records & Append them
while processed < len(proteins):
    print(f'Retrieving protein sequence for {proteins[current]}...')
    try:
        handle = Entrez.efetch(db='protein', id=proteins[current] , rettype='fasta', retmode='text')
        record = SeqIO.read(handle, 'fasta')
        records.append(record)
        processed += 1
        current += 1
    except:
        print(f'Retry after 5 seconds...')
        time.sleep(5)
        continue

# Save the protein records to a file
with open('insulin_family.fasta', 'w') as file:
    for record in records:
        file.write(record.format('fasta'))
print(f"\n✓ Data exported successfully.")

# Function to analyse a single protein
def analyse_protein(record):
    analysis = ProtParam.ProteinAnalysis(str(record.seq))
    properties = {
        'Protein ID': record.id,
        'Sequence Length': f"{analysis.length} amino acids",
        'Molecular Weight': f"{round(analysis.molecular_weight(), 4)} Da",
        'Isoelectric Point': round(analysis.isoelectric_point(), 4),
        'Instability Index': round(analysis.instability_index(), 4),
        'Aromaticity': round(analysis.aromaticity(), 4),
        'GRAVY (Hydrophobicity)': round(analysis.gravy(), 4),
        # You could add other properties like charge, helix/sheet fraction here
    }
    return properties
    
# Function to analyse multiple proteins
def analyse_multiple_proteins(insulin_fasta):
    records = SeqIO.parse(insulin_fasta, 'fasta')
    all_proteins_properties = []
    for record in records:
        protein_properties = analyse_protein(record)
        all_proteins_properties.append(protein_properties)
    properties_df = pd.DataFrame(all_proteins_properties)
    return properties_df

prot_family_analysis = analyse_multiple_proteins('insulin_family.fasta')
print("BATCH ANALYSIS COMPLETE: INSULIN FAMILY PROPERTIES")
print(prot_family_analysis.to_string(index=False))

Retrieving protein sequence for P01308...
Retrieving protein sequence for P01322...
Retrieving protein sequence for P01325...
Retrieving protein sequence for P01329...

✓ Data exported successfully.
BATCH ANALYSIS COMPLETE: INSULIN FAMILY PROPERTIES
            Protein ID Sequence Length Molecular Weight  Isoelectric Point  Instability Index  Aromaticity  GRAVY (Hydrophobicity)
 sp|P01308.1|INS_HUMAN 110 amino acids    11980.7866 Da             5.2186            40.3328       0.0818                  0.1927
  sp|P01322.1|INS1_RAT 110 amino acids    12420.3803 Da             5.6226            47.3674       0.0909                  0.0173
sp|P01325.1|INS1_MOUSE 108 amino acids    12160.0224 Da             5.5226            40.4640       0.0833                  0.0435
 sp|P01329.2|INS_CAVPO 110 amino acids     12186.945 Da             5.1470            59.1583       0.0727                  0.0109


# Bioinformatics II: Computational Analysis of Protein Data
## Tutorial Notebook - Protein Sequences, Structures, and Interaction Networks

**Learning Path:** Beginner → Intermediate → Advanced

---

### What You'll Learn Today:
1. Recap of **Part 1:** 
2. **Part 2 (Intermediate):** Understanding protein structures - PDB files and structural properties

---

### Recap of Part 1: Working with protein sequences

#### Getting the protein properties from sequence

Let's just start from the FASTA file downloaded from the Part I of the notebook.

In [ ]:
from Bio import SeqIO

seq_data = []

for record in SeqIO.parse('P01308.fasta', 'fasta'):
    seq_data.append({
        'id': record.id,
        'description': record.description,
        'sequence': str(record.seq),
        'length': len(record.seq)
    })

print(f'Data read successfuly!')
print(seq_data[0])

Data read successfuly!
{'id': 'sp|P01308.1|INS_HUMAN', 'description': 'sp|P01308.1|INS_HUMAN RecName: Full=Insulin; Contains: RecName: Full=Insulin B chain; Contains: RecName: Full=Insulin A chain; Flags: Precursor', 'sequence': 'MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN', 'length': 110}


There is one more functions in `BioPython` library that is very handy in getting the protein properties from the sequence, which is `ProtParam` module. 
1. Let's import the `ProtParam` sub-module from `SeqUtils` module in `BioPython`
2. then we will pass the sequence into the `ProteinAnalysis()` of `ProtParam`
3. This will produce an `ProtParam` object that we can use to generate different analysis.

In [ ]:
from Bio.SeqUtils import ProtParam

analysis = ProtParam.ProteinAnalysis(seq_data[0]['sequence'])

Using the `analysis` variable, which is an `ProtParam` object, we can do following by calling the functions built in for `ProtParam` object.

The following codes will get the following properties:
- `length`: the length of the sequence
- `molecular_weight()`: the total mass of the protein
- `isoelectric_point()`: the pH at which the protein has no net charge
- `instability_index()`: Instability index of protein, if <40 is stable, >40 is unstable
- `aromaticity()`: Relative frequency of aromatic amino acids
- `gravy()`: measure hyropathy of protein, negative = hydrophilic, positive = hydrophobic

In [3]:
print('Protein properties')
print('------------------')
print(f'Sequence: {analysis.sequence}')
print(f'Sequence length: {analysis.length}')
print(f'Molecular weight: {analysis.molecular_weight():.2f} Da')
print(f'pI: {analysis.isoelectric_point():.2f}')
print(f'Instability index: {analysis.instability_index():.2f}')
print(f'Aromaticity: {analysis.aromaticity():.2f}')
print(f'Gravy (hydropathy): {analysis.gravy():.2f}')

Protein properties
------------------
Sequence: MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN
Sequence length: 110
Molecular weight: 11980.79 Da
pI: 5.22
Instability index: 40.33
Aromaticity: 0.08
Gravy (hydropathy): 0.19


####
---
#### Now let's make a custom function that produce analysis for given protein sequence:

the function `analyse_protein()` below will take the protein sequence, and perform the `ProteinAnalysis()` and eventually retrieve the information including sequence, length, molecular weight, isoelectric point, instability index, aromaticity and gravy. lastly returned as a `dict` (dictionary) format.

In [4]:
def analyse_protein(sequence):
    analysis = ProtParam.ProteinAnalysis(sequence)
    properties = {
        'Sequence': analysis.sequence,
        'Length': analysis.length,
        'Molecular Weight': round(analysis.molecular_weight(),4),
        'Isoelectric Point': round(analysis.isoelectric_point(),4),
        'Instability Index': round(analysis.instability_index(),4),
        'Aromaticity': round(analysis.aromaticity(),4),
        'Gravy': round(analysis.gravy(),4)
    }
    return properties


# try to call the function
analyse_protein(seq_data[0]['sequence'])

{'Sequence': 'MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN',
 'Length': 110,
 'Molecular Weight': 11980.7866,
 'Isoelectric Point': 5.2186,
 'Instability Index': 40.3328,
 'Aromaticity': 0.0818,
 'Gravy': 0.1927}

####
---

### [PRACTICE] Use Case 2: Comparing Protein Family Members
**Biological Context:** Let's analyze multiple proteins from the insulin family to compare their properties.

**Data:** We'll create a small dataset of insulin-related proteins

- P01325 - insulin mouse
- P01308 - insulin human
- P01322 - insulin rat
- P01329 - insulin guinea pigs

1. Run the following code to create the small dataset that consists of four proteins above in Fasta format.

In [5]:
from Bio import Entrez
from Bio import SeqIO
import time

Entrez.email = 'nguyuling@graduate.utm.my'

proteins = ['P01308', 'P01322', 'P01325', 'P01329']

records = []

processed = 0
current   = 0

while processed < len(proteins):
    print(f'Retrieving protein sequence for {proteins[current]}...')
    try:
        handle = Entrez.efetch(db='protein', id=proteins[current] , rettype='fasta', retmode='text')
        record = SeqIO.read(handle, 'fasta')
        records.append(record)
        processed += 1
        current += 1
    except:
        print(f'Retry after 5 seconds...')
        time.sleep(5)
        continue

with open('insulin_family.fasta', 'w') as file:
    for record in records:
        file.write(record.format('fasta'))

print("Data exported successfully.")

Retrieving protein sequence for P01308...
Retrieving protein sequence for P01322...
Retrieving protein sequence for P01325...
Retrieving protein sequence for P01329...
Data exported successfully.


2. Then create a function called `analyse_multiple_proteins()` and pass the fasta file as the input parameter.
3. The function should follows the steps below:
   1. Read the FASTA file with multiple proteins and parse the proteins sequences into a variable called `sequences`
   2. For every protein sequence in the `sequences`, run the `analyse_protein()` function and get the properties.
   3. Then the properties returned will be combined into a result variable specific for each protein.
   4. Lastly combine all results from all proteins and convert into a `pandas` data frame and return it.

In [ ]:
def analyse_multiple_proteins(insulin_fasta):
    SeqIO.read(insulin_fasta)
    sequences = SeqIO.parse(insulin_fasta)
    
    for seqeunce in sequences:
        analyse_protein(sequence)
    
def analyse_protein(record):
    

In [5]:
import pandas as pd

def analyze_multiple_proteins(fasta_file):
    ## your code here

prot_family_analysis = analyze_multiple_proteins('insulin_family.fasta')
prot_family_analysis

IndentationError: expected an indented block after function definition on line 3 (792790863.py, line 6)

4. We could use this information to further provide visualizations to compare, study and analyze.

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(16,10))

# Plot 1 (upper left)
# Length comparison
axes[0,0].bar(prot_family_analysis['Protein'], prot_family_analysis['Length'])
axes[0,0].set_title("Comparison of Sequence Length")
axes[0,0].set_ylabel('Length (no. amino acids)')
axes[0,0].set_xlabel('Protein')
axes[0,0].set_ylim(0,max(prot_family_analysis['Length'])+10)
axes[0,0].tick_params(axis='x', rotation=45)

# Plot 2 (upper right)
# Molecular weight
axes[0,1].bar(prot_family_analysis['Protein'], prot_family_analysis['Molecular Weight'], color='purple')
axes[0,1].set_title("Comparison of Molecular Weights")
axes[0,1].set_ylabel('Molecular weights (Da)')
axes[0,1].set_xlabel('Protein')
axes[0,1].set_ylim(0,max(prot_family_`analysis['Molecular Weight'])+2000)
axes[0,1].tick_params(axis='x', rotation=45)

# Plot 3 (lower left)
# Isoelectric point
axes[1,0].bar(prot_family_analysis['Protein'], prot_family_analysis['Isoelectric Point'],color='orange')
axes[1,0].set_title("Comparison of Isoelectric point")
axes[1,0].set_ylabel('pH')
axes[1,0].set_xlabel('Protein')
axes[1,0].set_ylim(0,max(prot_family_analysis['Isoelectric Point'])+1)
axes[1,0].tick_params(axis='x', rotation=45)

# Plot 4 (lower right)
# Gravy
axes[1,1].bar(prot_family_analysis['Protein'], prot_family_analysis['Gravy'], color='green')
axes[1,1].set_title("Hydropathy (GRAVY)")
axes[1,1].set_ylabel('Score')
axes[1,1].set_xlabel('Protein')
axes[1,1].set_ylim(0,max(prot_family_analysis['Gravy'])+0.05)
axes[1,1].axhline(y=0, color='r', linestyle='--', linewidth=4)
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

What demonstrated above is one of the example how we can extract, compare different protein sequences on the properties of the proteins. 

####
---
## PART 2: # PART 2: Working with Protein Structures (Intermediate)

### 2.1 Understanding PDB Format

### Background:
The Protein Data Bank (PDB) is a database of 3D structural data of proteins. PDB files contain atomic coordinates in 3D space.

### Use Case 3: Analyzing Hemoglobin Structure
**Biological Context:** Hemoglobin carries oxygen in blood. Its 3D structure is crucial for function.

**Data Source:** RCSB PDB (https://www.rcsb.org/)
- Protein: Human Hemoglobin (PDB ID: 1A3N)
- Format: PDB file (text format with 3D coordinates)

### 2.2 Downloading PDB Files
Different with protein sequence, protein structures having more comprehensive information related to the structure of the proteins, location of the amino acids, secondary structure information and etc.

#### Method 1: using `requests`
The following example shows the way to download the protein structure from Protein Data Bank (RCSB PDB). Basically, the method uses `requests` and uses a standard URL to retrieve the structure file for specific proteins using teh 4-character PDB identifier.

In [ ]:
import requests

base_url = "https://files.rcsb.org/download/"
pdb_id   = "1A3N"
filename = pdb_id+".pdb"
req_url  = base_url+filename

response = requests.get(req_url)

if response.status_code==200:
    with open(filename,'w') as file:
        file.write(response.text)
    print(f'PDB file {filename} downloaded successfully.')
else:
    print(f'Failed to download. Status code: {response.status_code}')


#### Method 2: using `PDB` module in `Biopython`
The following example uses `PDB` module to retrieve the protein structure. However the standard file name are with the extension of `.ent`, but the information are as same as if retrieve using method 1.

In [ ]:
from Bio import PDB

pdb_id = "1A3N"  # Example PDB ID
download_directory = "." # <- . means download to the current folder
file_format = "pdb" 

pdb_list = PDB.PDBList()

pdb_filename = pdb_list.retrieve_pdb_file(pdb_id, pdir=download_directory, file_format=file_format, overwrite=True)
print(f"Downloaded PDB file: {pdb_filename}")

---
### 2.3 Parsing PDB Files
PDB files are text files with a specific format, but they're not immediately usable for computation. Parsing transforms raw structural data into computable objects.

Parsing creates a hierarchical data structure:
```
Structure
  └── Model (experimental structure, usually just one)
       └── Chain A, B, C, D... (protein subunits)
            └── Residue 1, 2, 3... (amino acids)
                 └── Atom N, CA, C, O... (individual atoms)
```

To parse the PDB files, we will use `PDBParser` function from `PDB` module in `Biopython`

In [ ]:
from Bio.PDB import PDBParser

source = 'pdb1a3n.ent' # the pdb file

parser = PDBParser(QUIET=True)
structure = parser.get_structure(pdb_id, source)

print("Structure Information:")
print("======================")
print(f"PDB ID: {structure.id}")
print(f"Number of models: {len(structure)}")

total_residues = 0
total_atoms = 0

for ind, model in enumerate(structure):
    print("\n----------------------")
    print(f"Structure for model {ind+1}")
    print("----------------------")
    print(f"Number of chains: {len(model)}")
    print("Per-Chain Information")
    for  chain in model:
        residues = [residue for residue in chain if residue.id[0]==' '] # checking the residue id if empty means standard amino acids
        atoms    = [atom for residue in residues for atom in residue]
        residue_count = len(residues)
        atom_count    = len(atoms)
        print(f'  Chain {chain.id}: {residue_count} residues, {atom_count} atoms')
        total_residues += residue_count
        total_atoms += atom_count

print(f'\nTotal residues: {total_residues}')
print(f'Total atoms: {total_atoms}')
print("----------------------")

---
#### [Practice]
Create a function called `parse_pdb(pdb_file)` that takes the PDB file as input. The function should return the structure parsed from the file, and will display the structure information as above.

In [ ]:
def parse_pdb(pdb_file):
    # your code here

    return structure

# your function should work with this eventually
ans = parse_pdb('1A3N.pdb')

---
### 2.4 Calculate Structural Properties

#### Center of Mass (COM)
The center of mass is like the average position of all the atoms, but weighted by their mass.
It tells you where the protein would “balance” if you could somehow place it on a tiny point in 3D space.

If all atoms had the same mass, then the center of mass would simply be the average of all (x, y, z) coordinates.
But since heavier atoms (like sulfur or oxygen) contribute more, we compute a weighted average instead.

The average position of all atoms, weighted by their mass:
```
COM = Σ(mass_i × position_i) / Σ(mass_i)
```

Example Pseudocode
```
1. for each chain in the model
2. for each residue in the chain
3.    check if the first letter in the residue id is '  ':
4.        for each atom in the residue
5.            get the mass of that atom
6.            get the coordinate of that atom
7.            calculate the weighted coordinates
8.            calculate the total mass
9.        end for
10    end if
11 end for
12 end for
```

In [ ]:
import numpy as np

model  = structure[0] #the structure get from cell above that reads the PDB file

total_mass = 0
weighted_coords = np.array([0.0, 0.0, 0.0])

# --------------
# Your code here





# --------------

center = weighted_coords / total_mass
print(center)

#### Radius of Gyration (Rg)
If the center of mass (COM) tells you where the protein is located in space,
the radius of gyration (Rg) tells you how spread out its atoms are around that center. In another words Rg measures how compact or extended the protein structure is.

Measure of how spread out the protein is from its center of mass:
```
Rg = √(Σ(mass_i × distance²_i) / Σ(mass_i))
```
where distance_i = distance from atom i to COM

##### **Physical Interpretation**
- **Small Rg** (compact): Protein is tightly packed
- **Large Rg** (extended): Protein is spread out
- **Intermediate Rg**: Typically folded but with flexible regions

Example Pseudocode
```
1. for each chain in the model
2. for each residue in the chain
3.    check if the first letter in the residue id is '  ':
4.        for each atom in the residue
5.            get the mass of that atom
6.            get the coordinate of that atom
7.            calculate the distance of the atom to from the center
8.            calculate the weighted distance
9.            calculate the total mass
9.        end for
10    end if
11 end for
12 end for
```

In [ ]:
model = structure[0]

total_mass = 0
weighted_dist = 0

# --------------
# Your code here





# --------------

rg = np.sqrt(weighted_dist / total_mass)
print(rg)

##### **alternative**
One also can use another library `neurosnap` to get the structure information for a protein (including COM, distance from COM, and more). Though Rg is not directly obtainable but can be calculated based on the distance info obtained from the library. 

Learn more: https://neurosnap.ai/blog/post/understanding-the-radius-of-gyration-in-protein-design-bioinformatics/66fb0ecc5a941760bf4b5138

In [ ]:
#!pip install neurosnap  # uncomment to install it.

In [ ]:
from neurosnap.protein import Protein

import warnings
warnings.filterwarnings('ignore')

protID = '1A3N'
prot_structure = Protein(protID)


print(f"Structure Information for {protID} using neurosnap")
print(f"Center of mass: {prot_structure.calculate_center_of_mass()}")
print(f"Distances from COM: {prot_structure.distances_from_com()}")

R_g = np.sqrt(np.sum(distances_from_com**2) / len(distances_from_com))
print(f"Radius of Gyration of the Protein: {R_g:.2f} Å")

---
### 2.5 Visualizing the protein using `py3Dmol`
Using `py3Dmol` we can view the 3D model of a given protein (PDB id)

In [ ]:
import py3Dmol

view = py3Dmol.view(query='pdb:1A3N')
view.setStyle({'cartoon':{'color':'spectrum'}})
view.show()

---